# Examples of discontinuities

::: {#cell-1 .cell 0=‘h’ 1=‘i’ 2=‘d’ 3=‘e’}

``` python
%load_ext autoreload
%autoreload 2
```

:::

In [ ]:
# /// pyproject
# run.requirements = [
#   "pyyaml",
#   "pydantic",
#   "pyspedas",
#   "pytplot",
#   "matplotlib",
#   "scienceplots",
# ]
# /

import sys
import yaml

from space_analysis.ds.tplot import export, process_panel
from utils import Config

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(["science", "nature", "notebook"])
plt.rc("savefig", dpi=300)
# plt.rc('svg', fonttype='none')

In [ ]:
file_path = sys.argv[1]

In [ ]:
config = yaml.load(open(file_path), Loader=yaml.FullLoader)
config = Config(**config)
tvars2plot = []

In [ ]:
for p_config in config.panels:
    tvar2plot = process_panel(p_config)
    tvars2plot.append(tvar2plot)

In [ ]:
# tlimit([ time.isoformat() for time in config.timerange])
export(config.output, tvars2plot)